# Plumed Analysis Notebook

Using this notebook to visualize and analyze the output from the H-bond metadynamics simulations

In [ ]:
import plumed
import signac
import signac_project
import pymbar
import matplotlib.pyplot as plt
import subprocess
import os
plt.rcParams.update({'font.size':9})
retcode = subprocess.call(['echo', 'foo'], 
    stdout=subprocess.DEVNULL,
    stderr=subprocess.STDOUT)

kt = 300 * 8.314462618 * 10 ** -3

## Get Signac setup

In [ ]:
project = signac.get_project()
project

In [ ]:
!python signac_project.py status

## Check CV of finished simulations

Using signac we can access simulations that have finished running

In [ ]:
# Funciton to plot H-bonds for individual directories
hide_plots = False

def plot_CV_bias(job, filename = "CV_bias_plot.png", dpi=300):
    plt.figure(dpi=dpi)
    data = plumed.read_as_pandas(job.fn("HBOND_SUMS"))
    fig, ax = plt.subplots(5,1, figsize = [12,8])
    ax[0].set_title("SIGMA: " +  str(job.sp.sigma) +  " HEIGH:" + str(job.sp.height))
    ax[0].plot(data.time/1000, data.values[:,1])
    ax[0].set_ylabel("Hydrogen Bond 1")
    ax[1].plot(data.time/1000, data.values[:,2])
    ax[1].set_ylabel("Hydrogen Bond 2")
    ax[2].plot(data.time/1000, data.values[:,3])
    ax[2].set_ylabel("Hydrogen Bond 3")
    ax[3].plot(data.time/1000, data.values[:,4])
    ax[3].set_ylabel("Sum of H-Bonds")
    ax[4].plot(data.time/1000, data.values[:,5],'r')
    ax[4].set_ylabel("Bias Energy")
    ax[4].set_xlabel("Time (ns)")
    if hide_plots:
        plt.close()
    plt.savefig(job.fn(filename))

In [ ]:
for job in project.find_jobs():
    if signac_project.check_production_npt_finish(job):
        plot_CV_bias(job)  

## Run preliminary free energy calculation

First we can calculate a preliminary FES using the `plumed sum_hills` function.

In [ ]:
os.path.abspath("")

In [ ]:
current_dir = os.path.abspath("")

In [ ]:
for job in project.find_jobs():
    if signac_project.check_production_npt_finish(job):
        os.chdir(job.fn(""))
        subprocess.run(["plumed", "sum_hills", "--hills", "HILLS", "--min", "0", "--max", "3", "--kt", str(kt)])
        fes_data = plumed.read_as_pandas("fes.dat")
        plt.figure()
        plt.plot(fes_data.values[:,0], fes_data.values[:,1])
        plt.title("SIGMA: " + str(job.sp.sigma) + " HEIGHT: " + str(job.sp.height))
        plt.xlabel("Number of Hydrogen Bonds")
        plt.ylabel("Free Energy")
        os.chdir(current_dir)

In [ ]:
def calculate_statistical_inefficiency():
    pass

In [ ]:
for job in project.find_jobs():
    if signac_project.check_production_npt_finish(job):
        # Calculate